In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk import ne_chunk, pos_tag
from nltk import Tree
from nltk.corpus import stopwords
import uuid
import sqlite3 #database bonus task

pd.set_option('display.max_colwidth', None)

# TASK 1: Scraping

In [2]:
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()))

In [45]:
def find_products(url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    product_data = []
    
    for _ in range(5):  # Loop through 5 pages
        ad_cards = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'info--ifj7U')))
        
        for i in range(len(ad_cards)):  # Use a range to avoid stale element references
            ad_cards = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'info--ifj7U')))
            card = ad_cards[i]
            
            # Check for the DarazMall status element
            dmall_elements = card.find_elements(By.XPATH, './/i[contains(@class, "ic-dynamic-badge")]')
            dmall_status = any("lazMall" in elem.get_attribute("class") for elem in dmall_elements)

            product_info = {}
            product_title_element = card.find_element(By.CLASS_NAME, 'title--wFj93')
            try:
                product_info['Product Title'] = product_title_element.text
            except Exception as e:
                product_info['Product Title'] = 'N/A'
                
            product_info['Link'] = product_title_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
            product_info['Price'] = card.find_element(By.CLASS_NAME, 'price--NVB62').text
            product_info['Daraz Mall'] = dmall_status

            # Calculate product rating based on stars
            try:
                stars = card.find_elements(By.XPATH, ".//div[contains(@class, 'rating--ZI3Ol')]//i[contains(@class, 'star-icon--k88DV')]")
                rating = sum('star-10' in star.get_attribute('class') for star in stars) + \
                         sum('star-9' in star.get_attribute('class') for star in stars) + \
                         sum('star-8' in star.get_attribute('class') for star in stars) + \
                         sum('star-7' in star.get_attribute('class') for star in stars) + \
                         sum('star-6' in star.get_attribute('class') for star in stars) + \
                         sum('star-5' in star.get_attribute('class') for star in stars) * 0.5
                product_info['Rating'] = rating
            except:
                product_info['Rating'] = 'N/A'

            # Check for shipping status
            try:
                product_shipstat_element = card.find_element(By.XPATH, ".//span[contains(@class, 'location--eh0Ro')]")
                product_info['Shipping Status'] = product_shipstat_element.text
            except:
                product_info['Shipping Status'] = 'N/A'

            product_data.append(product_info)

        # Navigate to the next page
        try:
            next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//li[contains(@class, "ant-pagination-next") and not(contains(@class, "ant-pagination-disabled"))]')))
            next_page_button.click()
        except Exception as e:
            print("Error clicking next page:", e)
            break

    # Now that all elements on the first page are processed, you can safely move to the next page
    for i, product in enumerate(product_data): # Assigning unique ID
        # Generate a unique ID for each product
        product['Product ID'] = str(uuid.uuid4())
        # Extract brand from product title
        product['Brand'] = product['Product Title'].split()[0] if product['Product Title'] else 'Unknown'

    # Filter out irrelevant products
    product_data = [product for product in product_data if 
                    not any(keyword in product['Product Title'].lower() 
                            for keyword in ['case', 'cover', 'charger'])]

    # Fetch additional details from each product's page
    for product in product_data:
        driver.get(product['Link'])
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'pdp-product-title')))
        
        try:
            product_seller_rating = driver.find_element(By.CLASS_NAME, 'seller-info-value.rating-positive').text
        except:
            product_seller_rating = 'N/A'
        try:    
            product_ship_exp_elements = driver.find_elements(By.CLASS_NAME, 'seller-info-value')
            product_ship_exp = product_ship_exp_elements[1].text if len(product_ship_exp_elements) > 1 else 'N/A'
        except:
            product_ship_exp = 'N/A'
        
        product['Seller Rating'] = product_seller_rating
        product['Ship On Time'] = product_ship_exp

    return pd.DataFrame(product_data), product_data

def clean_df(df):
    # Replace 'N/A' with NaN and '%' with nothing, then convert to float
    df['Seller Rating'] = df['Seller Rating'].replace('N/A', np.nan).replace('%', '', regex=True).astype(float)
    # Convert Price to float after removing 'Rs.' and commas
    df['Price'] = df['Price'].replace('Rs. ', '', regex=True).replace(',', '', regex=True).astype(float)
    df['Shipping Status'] = df['Shipping Status'].replace('Pakistan','No Free Shipping')
    return df

def get_reviews(url, product_id):
    driver.get(url)
    temp = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="module_product_review"]')))
    except TimeoutException:
        return temp  # Return an empty list if the reviews section is not found within the timeout

    PAGE_LIMIT = 5
    for page in range(PAGE_LIMIT):
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'review-item')))
        except:
            break  # Exit the loop if no reviews are found

        reviews = driver.find_elements(By.CLASS_NAME, 'review-item')

        for review in reviews:
            try:
                review_text = review.find_element(By.CLASS_NAME, 'review-content-sl').text
                if review_text:
                    temp.append({'Product ID': product_id, 'Review': review_text})
            except NoSuchElementException:
                continue

        try:
            next_button_xpath = "//li[contains(@class, 'ant-pagination-next') and not(@aria-disabled='true')]/button"
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
            if next_button:
                next_button.click()
                time.sleep(5)
            else:
                break
        except:
            break  # Exit the loop if the next button is not found

    return temp

def get_sentiments(reviews):
    sia = SentimentIntensityAnalyzer()
    review_texts = [item.get('Review', 'No review text') for sublist in reviews for item in sublist if isinstance(item, dict)]
    sentiments = [sia.polarity_scores(review) for review in review_texts]
    return sentiments

def classify_sentiment(row):
    compound = row['Sentiments']['compound']
    if compound > 0.05:
        return 'Positive'
    elif compound < -0.05:
        return 'Negative'
    else:
        return 'Neutral'
    
def create_review_sentiment_df(reviews):
    # Flatten the reviews list
    flattened_reviews = [review for sublist in reviews for review in sublist]
    df_reviews = pd.DataFrame({
        'Product ID': [review['Product ID'] for review in flattened_reviews],
        'Review': [review['Review'] for review in flattened_reviews],
        'Sentiments': get_sentiments(reviews)
    })

    df_reviews['Review Sentiment'] = df_reviews.apply(classify_sentiment, axis=1)
    df_reviews = df_reviews.drop('Sentiments', axis=1)
    return df_reviews

In [15]:
url = 'https://www.daraz.pk/smartphones/'
products_df,links = find_products(url)
products_df = clean_df(products_df)

In [30]:
reviews = [get_reviews(item['Link'], item['Product ID']) for item in links]

In [46]:
df_reviews = create_review_sentiment_df(reviews)

In [47]:
products_df

,Product Title,Link,Price,Daraz Mall,Rating,Shipping Status,Product ID,Brand,Seller Rating,Ship On Time
0,Infinix Note 30 Pro 8-256 GB PTA Approved With Official 1 Year Warranty,https://www.daraz.pk/products/infinix-note-30-pro-8-256-gb-pta-approved-with-official-1-year-warranty-i436821010-s2101334479.html?search=1,64499.0,False,5.0,Free Shipping,1b38eec4-33b1-4863-b877-c13fa3693b50,Infinix,NaN,Not enough data
1,Realme Narzo 50A Prime (4GB-128GB) PTA Approved With Official 1 Year Warranty,https://www.daraz.pk/products/realme-narzo-50a-prime-4gb-128gb-pta-approved-with-official-1-year-warranty-i436801398-s2101274808.html?search=1,32999.0,False,4.0,Free Shipping,06cb1956-5b14-4d7e-8018-b8b98029c14b,Realme,NaN,Not enough data
2,vivo Y17s - 6GB RAM - 128GB ROM - 5000mAh Battery - 50+2 MP Camera - Helio G85 Processor,https://www.daraz.pk/products/vivo-y17s-6gb-ram-128gb-rom-5000mah-battery-502-mp-camera-helio-g85-processor-i434216463-s2082615909.html?search=1,44999.0,True,5.0,Free Shipping,795c4ac5-e2f6-4eed-ae5f-adce9c159068,vivo,92.0,48%
3,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approved Mobile Phone with 1 Year Brand Warranty,https://www.daraz.pk/products/redmi-note-12-8-gb-ram-128gb-rom-pta-approved-mobile-phone-with-1-year-brand-warranty-i429185340-s2040915791.html?search=1,64999.0,True,5.0,No Free Shipping,8bf4b339-0446-4e16-a587-fedd68ab6ba8,Redmi,92.0,100%
4,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - Android 12- 5000 mAh Battery,https://www.daraz.pk/products/redmi-a2-3gb64gb-mediatek-helio-g36-android-12-5000-mah-battery-i430451931-s2051820850.html?search=1,24999.0,True,5.0,No Free Shipping,c1e39f9f-82f9-41d7-b34b-41b9ea9fbaf6,Redmi,92.0,100%
...,...,...,...,...,...,...,...,...,...,...
188,ITEL S23 16GB* Customizable RAM 256GB ROM; Expandable upto 1TB PTA APPROVED OFFICIAL BRAND WARRANTY,https://www.daraz.pk/products/itel-s23-16gb-customizable-ram-256gb-rom-expandable-upto-1tb-pta-approved-official-brand-warranty-i435965776-s2094070045.html?search=1,33150.0,True,0.0,Free Shipping,f8002d58-7557-4f4e-9155-7e2a628f1672,ITEL,85.0,76%
189,MOTO G6 SINGLE SIM ONLY KIT,https://www.daraz.pk/products/g6-i422443620-s1998808828.html?search=1,17500.0,False,4.5,Free Shipping,b716d9c5-e25d-47c5-ab36-9d9faea2f704,MOTO,94.0,100%
190,Samsung Galaxy A04 || 3GB Ram 32GB Rom || 6.5 Inches IPS Display || 5000 mAh - Battery charging 7.8W,https://www.daraz.pk/products/samsung-galaxy-a04-3gb-ram-32gb-rom-65-inches-ips-display-5000-mah-battery-charging-78w-i396467493-s1921803740.html?search=1,25199.0,False,0.0,Free Shipping,8934f2cf-d11a-49ed-a9da-24e1c0fed31a,Samsung,90.0,67%
191,ZTE A53 PRO - 4GB RAM 64GB ROM - Dual SIM - 5000mAh Battery,https://www.daraz.pk/products/zte-a53-4gb-64gb-5000mah-i433403543-s2080673503.html?search=1,21499.0,False,0.0,Free Shipping,5b4f5241-b466-4ba3-9a41-f273c758b00c,ZTE,89.0,92%


In [48]:
df_reviews

,Product ID,Review,Review Sentiment
0,1b38eec4-33b1-4863-b877-c13fa3693b50,"I got what was shown, but I want to tell about the seller that my chat was not responded even after a couple of days, so please make your availability there so that the customer can place the order with full sitesify. Because the amount matters But my goal was to buy the product from there, so I took what I have and it is 💯 %authentic, and the warranty has not started yet, But the handsfree did not come out of the box, maybe not provided by the company. So the Overall experience was great and the seller is highly recommended. Thanks Daraz & Seller.🥰",Positive
1,1b38eec4-33b1-4863-b877-c13fa3693b50,Great Experience as usual and I have purchased many mobiles from DARAZ but this time get maximum discount in 11/11 Sales on Note 30 pro,Positive
2,1b38eec4-33b1-4863-b877-c13fa3693b50,Allahamdolilah received📩 mobile is original and new as mentioned recommended for buying. same thing which is ordered rather late delivered,Positive
3,1b38eec4-33b1-4863-b877-c13fa3693b50,"Acha mobile hai and same cheez delivered ki hai with fast delivery service, satisfied with daraz and seller, i bought many mobiles from daraz aj tak koi b mobile kharab nhe nikla, thanks daraz for maintenance of trust with customers. love from Rawalpindi",Positive
4,1b38eec4-33b1-4863-b877-c13fa3693b50,Allahamdolilah received📩 mobile is original and new as mentioned recommended for buying. same thing which is ordered rather late delivered,Positive
...,...,...,...
956,b716d9c5-e25d-47c5-ab36-9d9faea2f704,👍👍,Neutral
957,b716d9c5-e25d-47c5-ab36-9d9faea2f704,Mashallah very best mobile phone seller reply every msg and very fantastic battery timing is also good. 5/5 star🌟🌟🌟🌟📱,Positive
958,f58c73ff-81b3-464a-b8c4-924ec904aba4,#11.11 The product was very good it was pin packed no scratch best product very smooth everything works great,Positive
959,f58c73ff-81b3-464a-b8c4-924ec904aba4,very nice and quick service. I have recived my parcel in less than a week A14. now i have order another mobile A04s. i hope this time again they provide nice service. Insha Allah,Positive


In [49]:
products_df.to_csv('products.csv', index=False)
df_reviews.to_csv('reviews.csv', index=False)

# TASK 2: Database Integration (Bonus)

In [3]:
products_df = pd.read_csv('products.csv')
df_reviews = pd.read_csv('reviews.csv')

In [4]:
def insert_products(df):
    conn = sqlite3.connect('products_reviews.db')
    cursor = conn.cursor()

    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO Products (ProductID, Name, Link, Price, DarazMallStatus, Rating, ShippingStatus, Brand, SellerRating, ShipOnTime)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
            (row['Product ID'], row['Product Title'], row['Link'], row['Price'], 
             row['Daraz Mall'], row['Rating'], row['Shipping Status'], 
             row['Brand'], row['Seller Rating'], row['Ship On Time']))

    conn.commit()
    conn.close()
    
def insert_reviews(df):
    conn = sqlite3.connect('products_reviews.db')
    cursor = conn.cursor()

    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO Reviews (ProductID, ReviewText, ReviewSentiment)
            VALUES (?, ?, ?)''',
            (row['Product ID'], row['Review'], row['Review Sentiment']))

    conn.commit()
    conn.close()
    
def view_table(table_name):
    # Connect to SQLite database
    conn = sqlite3.connect('products_reviews.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    df = pd.DataFrame(rows, columns=column_names)
    conn.close()

    return df

def get_products_in_price_range(min_price, max_price):
    conn = sqlite3.connect('products_reviews.db')
    query = '''
        SELECT * FROM Products
        WHERE Price BETWEEN ? AND ?
    '''
    df = pd.read_sql_query(query, conn, params=(min_price, max_price))
    conn.close()
    return df

def get_average_rating_by_brand():
    conn = sqlite3.connect('products_reviews.db')
    query = '''
        SELECT Brand, AVG(Rating) as AverageRating FROM Products
        GROUP BY Brand
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def get_reviews_for_top_rated_product():
    conn = sqlite3.connect('products_reviews.db')
    query = '''
        SELECT p.Name, p.ProductID, p.Rating, r.ReviewText, r.ReviewSentiment 
        FROM Products p
        INNER JOIN Reviews r ON p.ProductID = r.ProductID
        WHERE p.Rating = (SELECT MAX(Rating) FROM Products)
        AND p.ProductID = (SELECT ProductID FROM Products ORDER BY Rating DESC, ProductID LIMIT 1)
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

In [5]:
# Connect to SQLite database
conn = sqlite3.connect('products_reviews.db')
cursor = conn.cursor()

#Creating db

# Create table - Products
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Products (
        ProductID TEXT PRIMARY KEY, 
        Name TEXT, 
        Link TEXT, 
        Price REAL, 
        DarazMallStatus BOOLEAN,
        Rating REAL,
        ShippingStatus TEXT,
        Brand TEXT,
        SellerRating REAL,
        ShipOnTime TEXT)
''')

# Create table - Reviews
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Reviews (
        ReviewID INTEGER PRIMARY KEY AUTOINCREMENT, 
        ProductID TEXT, 
        ReviewText TEXT, 
        ReviewSentiment TEXT,
        FOREIGN KEY(ProductID) REFERENCES Products(ProductID))
''')

# Commit our changes
conn.commit()
conn.close()

In [6]:
#Inserting into db
insert_products(products_df)
insert_reviews(df_reviews)

In [7]:
#Retrieving from db
db_products = view_table("Products") #getting stored products from db
db_reviews = view_table("Reviews") #getting stored reviews from db

### Showing robustness of Database schema structure by quering data on various conditions

In [55]:
# Example usage
get_products_in_price_range(20000, 40000).head(2)

,ProductID,Name,Link,Price,DarazMallStatus,Rating,ShippingStatus,Brand,SellerRating,ShipOnTime
0,06cb1956-5b14-4d7e-8018-b8b98029c14b,Realme Narzo 50A Prime (4GB-128GB) PTA Approved With Official 1 Year Warranty,https://www.daraz.pk/products/realme-narzo-50a-prime-4gb-128gb-pta-approved-with-official-1-year-warranty-i436801398-s2101274808.html?search=1,32999.0,0,4.0,Free Shipping,Realme,NaN,Not enough data
1,c1e39f9f-82f9-41d7-b34b-41b9ea9fbaf6,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - Android 12- 5000 mAh Battery,https://www.daraz.pk/products/redmi-a2-3gb64gb-mediatek-helio-g36-android-12-5000-mah-battery-i430451931-s2051820850.html?search=1,24999.0,1,5.0,No Free Shipping,Redmi,92.0,100%


In [56]:
get_average_rating_by_brand() #we utilize groupby aggregation of SQL

,Brand,AverageRating
0,A57,0.000000
1,Apple,2.500000
2,Aquos,5.000000
3,Combo,4.000000
4,Galaxy,3.000000
5,Google,5.000000
6,HONOR,NaN
7,Honor,5.000000
8,INFINIX,4.666667
9,ITEL,0.000000


In [57]:
get_reviews_for_top_rated_product().head(5)

,Name,ProductID,Rating,ReviewText,ReviewSentiment
0,"Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable",030280c8-56f6-41cc-8b6b-2388869213db,5.0,"100 % Genuine Product , Too fast delivery by seller , Fast seller response perfectly packed by daraz I really enjoy the super fast and safe delivery by daraz Thanks Daraz Thanks sid sad communication seller",Positive
1,"Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable",030280c8-56f6-41cc-8b6b-2388869213db,5.0,"waited for Redmi Note 12 Pro's release at Xiaomi official store, but found no confirmation. Opted for this seller, initially seeking grey, only white and blue were available. Seller was kind & arranged the grey variant. Grateful for their responsiveness and a quick 2-day delivery, increased trust in Daraz with upfront debit card payment. arrived properly sealed, PTA approved. Impressed by fast charging, phone responsiveness, clear call quality, and excellent camera results. Highly recommended 😇",Positive
2,"Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable",030280c8-56f6-41cc-8b6b-2388869213db,5.0,"Brand new packed phone. Delivery was a bit slow but reasonable, received in 3 days. Value for money. PTA verified phone.",Positive
3,"Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable",030280c8-56f6-41cc-8b6b-2388869213db,5.0,"Brand new packed phone. Delivery was a bit slow but reasonable, received in 3 days. Value for money. PTA verified phone.",Positive
4,"Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB Front Camera 16 MP Back Camera 108 MP + 08 MP + 02 MP Battery Type Li-Po 5000 mAh, non-removable",030280c8-56f6-41cc-8b6b-2388869213db,5.0,Super Sonic fast Delivery from Seller. I just asked him that i need it fast and he keep his promise and delivered it on time. Other than that white colour is so much classic. Recommended Seller,Positive


# TASK 3: Chatbot

In [58]:
# in app.py

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Katrina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Katrina\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Katrina\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Katrina\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Katrina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# TASK 4: Dashboard (Using Flask)

In [54]:
# in app.py